## Delta Lake Format Usage

Delta Lake is the default format for tables in Databricks. You can create and write to Delta tables without explicitly specifying the format:

* Create table: `CREATE TABLE foo` ~~using delta~~
* DataFrame write: `df.write.saveAsTable("foo")`~~.format("delta")~~

In [ ]:
from pyspark.sql.types import *

dt1 = (
    DeltaTable.create(spark)
    .tableName("testTable1")
    .addColumn("c1", dataType="INT", nullable=False)
    .addColumn("c2", dataType=IntegerType(), generatedAlwaysAs="c1 + 1")
    .partitionedBy("c1")
    .execute()
)

## Data Loading with COPY INTO

The COPY INTO command provides efficient and reliable data loading into Delta tables:

In [ ]:
%%sql
-- Basic COPY INTO with parquet files
COPY INTO foo FROM 's3://bucket/path/to/file'
FILEFORMAT = parquet
COPY_OPTIONS = {
    'overwrite': True,
    'maxfiles': 1000,
    'mergeSchema': 'true'
};

-- COPY INTO with file name tracking
COPY INTO foo FROM
(SELECT *, input_file_name() FROM 's3://bucket/path/to/file')
FILEFORMAT = parquet
COPY_OPTIONS = {
    'overwrite': True
};


### CLONE

In [ ]:
%%sql
    CREATE TABLE db.target_table SHALLOW CLONE delta.`/data/source` VERSION AS OF version

    CREATE TABLE db.target_table SHALLOW CLONE delta.`/data/source` TIMESTAMP AS OF timestamp_expression

    CREATE OR REPLACE TABLE db.target_table SHALLOW CLONE db.source_table

    CREATE TABLE IF NOT EXISTS delta.`/data/target/` SHALLOW CLONE db.source_table

    # UNITY CATALOG
    CREATE TABLE <catalog-name>.<schema-name>.<target-table-name> SHALLOW CLONE <catalog-name>.<schema-name>.<source-table-name>

In [ ]:
    spark.sql('CREATE TABLE delta.`/data/target/` SHALLOW CLONE delta.`/data/source/` ')